# SfM

In [46]:
import os

%load_ext autoreload
%autoreload 2

src_path = os.getcwd()
print(src_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/unbinilium/Documents/Develop/Python/SfM/src


## Common Configs

In [47]:
datasets_path = '../data/datasets'
datasets_name = 'oreo'

image_dir_path = os.path.join(src_path, datasets_path, datasets_name, 'images')
print(image_dir_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/images


## Generate Masks

In [2]:
import generate_masks_and_masked_images

Running inference on device 'cpu'


/Users/unbinilium/Documents/Develop/Python/SfM/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [7]:
image_masks_path = os.path.join(src_path, datasets_path, datasets_name, 'masks')
masked_image_path = os.path.join(src_path, datasets_path, datasets_name, 'masked_images')
print(image_masks_path, masked_image_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/masks /Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/masked_images


In [8]:
generate_masks_and_masked_images.main(image_dir_path, image_masks_path, masked_image_path)

100%|██████████| 431/431 [17:15<00:00,  2.40s/it]


## Extract Features

In [48]:
import extract_features

In [49]:
superpoint_conf = {
    'output': 'feats-spp',
    'model': {
        'path': 'thirdparty/SuperPointPretrainedNetwork/superpoint_v1.pth',
        'name': 'spp_det'
    },
    'preprocessing': {
        'grayscale': True
    },
    'conf': {
        'descriptor_dim': 256,
        'nms_radius': 3,
        'max_keypoints': 4096,
        'keypoints_threshold': 0.6
    }
}

features_out_path = os.path.join(src_path, datasets_path, datasets_name, superpoint_conf['output'] + '.h5')
print(features_out_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/feats-spp.h5


In [50]:
extract_features.main(image_dir_path, features_out_path, superpoint_conf)

=> load weights:  thirdparty/SuperPointPretrainedNetwork/superpoint_v1.pth


  0%|          | 0/431 [00:00<?, ?it/s]/Users/unbinilium/Documents/Develop/Python/SfM/lib/python3.10/site-packages/torch/nn/functional.py:4227: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|██████████| 431/431 [02:47<00:00,  2.57it/s]


## Generate Pairs

In [51]:
import generate_pairs

In [52]:
generate_pairs_conf = {
    'method': 'from-poses',
    'num_matched': 10,
    'min_rotation': 10
}

pairs_out_path = os.path.join(
    src_path,
    datasets_path,
    datasets_name,
    '{}-pairs.txt'.format(generate_pairs_conf['method']))
print(pairs_out_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/from-poses-pairs.txt


In [53]:
generate_pairs.main(image_dir_path, pairs_out_path, generate_pairs_conf)

## Matching Features

In [54]:
import match_features

In [55]:
match_features_conf = {
    'model': {
        'path': 'thirdparty/SuperGluePretrainedNetwork/models/weights/superglue_indoor.pth'
    },
    'output': 'matches-spg',
    'conf': {
        'descriptor_dim': 256,
        'weights': 'indoor',
        'match_threshold': 0.7
    }
}

matches_out_path = os.path.join(
    src_path,
    datasets_path,
    datasets_name,
    '{}.h5'.format(match_features_conf['output']))
print(matches_out_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/matches-spg.h5


In [56]:
match_features.main(features_out_path, pairs_out_path, matches_out_path, match_features_conf)

Loaded SuperGlue model ("indoor" weights)
=> load weights:  thirdparty/SuperGluePretrainedNetwork/models/weights/superglue_indoor.pth


100%|██████████| 2155/2155 [09:31<00:00,  3.77it/s]


## Generate Empty

In [59]:
import generate_empty

In [60]:
sfm_empty_path = os.path.join(src_path, datasets_path, datasets_name, 'sfm_empty')
print(sfm_empty_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/sfm_empty


In [61]:
generate_empty.main(image_dir_path, sfm_empty_path)

## Triangulation

In [62]:
import triangulation

In [63]:
deep_sfm_path = os.path.join(src_path, datasets_path, datasets_name, 'sfm_ws')
sfm_outputs_path = os.path.join(src_path, datasets_path, datasets_name,
    'outputs' + '_' + superpoint_conf['output'] + '_' + match_features_conf['output'])

print(deep_sfm_path, sfm_outputs_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/sfm_ws /Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/oreo/outputs_feats-spp_matches-spg


In [64]:
triangulation.main(
    deep_sfm_path,
    sfm_empty_path,
    sfm_outputs_path,
    pairs_out_path,
    features_out_path,
    matches_out_path,
    image_dir=None
)

100%|██████████| 2155/2155 [00:00<00:00, 9653.89it/s]
